<a href="https://colab.research.google.com/github/ksawczak/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install eli5

# New Section

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [36]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [58]:
df = pd.read_csv('data/shoes_prices.csv', low_memory = False)
df.shape

(18280, 48)

In [0]:
#df = pd.read_csv('data/shoes_prices.csv', low_memory=False)
#df_usd = df[df.prices_currency == 'USD'].copy()
#df_usd['prices_amountmin'] = df_usd.prices_amountmin.astype(np.float)
#filter_max = np.percentile(df_usd['prices_amountmin'],99)
#df_usd_filter = df_usd[df_usd['prices_amountmin']<filter_max]

In [59]:
#df = df_usd_filter
df.shape

(18280, 48)

In [0]:
#df.to_csv('data/shoes_prices.csv', index=False)
#df = pd.read_csv('data/shoes_prices.csv', low_memory=False)
#df.shape

In [0]:
def run_model (feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [62]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [63]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [64]:
df.features

0        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2        [{"key":"Gender","value":["Men"]},{"key":"Colo...
3        [{"key":"Gender","value":["Men"]},{"key":"Colo...
4        [{"key":"Gender","value":["Men"]},{"key":"Colo...
                               ...                        
18275    [{"key":"Style","value":["Desert Boots"]},{"ke...
18276    [{"key":"Shoe Size","value":["9.5"]},{"key":"G...
18277    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18278    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18279    [{"key":"Gender","value":["Unisex"]},{"key":"F...
Name: features, Length: 18280, dtype: object

Start here

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item ['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df ['features_parsed'] = df['features'].map(parse_features)

In [66]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [67]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [68]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan )

In [0]:
keys_stat= {}
for key in keys:
  keys_stat[key] = df[ False  == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0] * 100

In [70]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_gender_cat'] = df['feat_color'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [72]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 1002)

In [73]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.35389966693642, 4.1377956448686914)

In [74]:
feats_cat = [x for  x in df.columns if 'cat' in x]
feats_cat


['categories',
 'brand_cat',
 'feat_clothing category',
 'feat_shoe category',
 'feat_location - city/state',
 'feat_certifications and listings',
 'feat_location - country',
 'feat_fabrication',
 'feat_catalog',
 'feat_multi pack indicator',
 'feat_recommended location',
 'feat_color_cat',
 'feat_brand_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_animal type_cat',
 'feat_nike id_cat',
 'feat_clothing size type_cat',
 'feat_number of pieces_cat',
 'feat_sports league_cat',
 'feat_style:_cat',
 'feat_reinforced pockets_cat',
 'feat_technician_cat',
 'feat_flame resistant_cat',
 'feat_lens material_cat',
 'feat_model_cat',
 'feat_removable liner_cat',
 'feat_ean_cat',
 'feat_bridge size:_cat',
 'feat_model no._cat',
 'feat_product dimensions_cat',
 'feat_made in_cat',
 'feat_elastic banded waist_cat',
 'feat_mirrored_cat',
 'feat_casual & dress shoe style_cat',
 'feat_shoe height_cat',
 'feat_toe type_c

In [0]:
feats = ['brand_cat', 'feat_metal type_cat','feat_brand_cat', 'feat_shape_cat' ,'feat_gender_cat',  'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats,model)

In [76]:
X = df[feats].values
y  = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names = feats)

(-57.28155559471959, 4.181600524423073)


Weight,Feature
0.2576 ± 0.0077,brand_cat
0.1031 ± 0.0075,feat_material_cat
0.0244 ± 0.0024,feat_gender_cat
0.0182 ± 0.0023,feat_brand_cat
0.0126 ± 0.0007,feat_shape_cat
0.0093 ± 0.0015,feat_metal type_cat
0.0031 ± 0.0004,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [77]:
df['brand'].value_counts(normalize = True)

nike                       0.097210
puma                       0.033315
ralph lauren               0.028775
vans                       0.021116
new balance                0.020295
                             ...   
king will                  0.000055
airwalk                    0.000055
w.k                        0.000055
rcs                        0.000055
u.s. polo assn. classic    0.000055
Name: brand, Length: 1732, dtype: float64

In [78]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.